In [9]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u 
from math import radians, degrees, sin, cos, asin, acos, sqrt
from opencage.geocoder import OpenCageGeocode
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import requests
from geopy.distance import great_circle
import pickle

plt.style.use("../harsh")

In [22]:
def gc(lon1, lat1, lon2, lat2):
    return great_circle((lat1, lon1), (lat2, lon2)).kilometers

vc_great_circle = np.vectorize(gc)

In [3]:
def carbon_simple_air_travel(travel_distance, travel_class="econ"):
    """
    simple calculator based on https://pics.uvic.ca/sites/default/files/AirTravelWP_FINAL.pdf. 
    takes in travel_distance [km, float] and travel_class [string]
    returns carbon emissions in [kg]
    """
    scaled_distance = 1.08*travel_distance #account for rerouting, holds, etc.
    
    if(scaled_distance < 463):
        return 0.27867*scaled_distance
    
    elif(scaled_distance < 3700):
        if(travel_class == "econ"):
            return 0.16508*scaled_distance
        else: 
            return 0.24761*scaled_distance 
    
    else:
        if(travel_class == "econ"):
            return 0.14678*scaled_distance
        elif(travel_class == "econ_plus"):
            return 0.23484*scaled_distance
        else: 
            return 0.42565*scaled_distance

In [4]:
def return_latlon(place):
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s'%(place, gmaps_api_key))
    resp_json_payload = response.json()
    results = resp_json_payload['results']
    if(len(results) == 0):
        return None
    else:
        lat = results[0]['geometry']['location']['lat']
        lng = results[0]['geometry']['location']['lng']
        return lat, lng

In [5]:
df = pd.read_csv("iau.csv")
df = df.fillna(" ")
inst = np.array(df['Institute'].values, dtype='str')
country = np.array(df['Country'].values, dtype='str')

In [6]:
unique_insts, idxs, counts = np.unique(inst, return_counts=True, return_index=True)

In [7]:
gmaps_api_key = "YOUR_API_KEY"
"""
fails = 0 
all_inst_latlons = np.zeros((len(unique_insts), 2))
for i in tqdm(range(len(unique_insts))):
    try:
        all_inst_latlons[i,0], all_inst_latlons[i,1] = return_latlon(unique_insts[i])
    except:
        print("%s DOESN'T EXIST" % unique_insts[i])
        fails += 1
        all_inst_latlons[i,0], all_inst_latlons[i,1] = np.nan, np.nan
"""


'\nfails = 0 \nall_inst_latlons = np.zeros((len(unique_insts), 2))\nfor i in tqdm(range(len(unique_insts))):\n    try:\n        all_inst_latlons[i,0], all_inst_latlons[i,1] = return_latlon(unique_insts[i])\n    except:\n        print("%s DOESN\'T EXIST" % unique_insts[i])\n        fails += 1\n        all_inst_latlons[i,0], all_inst_latlons[i,1] = np.nan, np.nan\n'

In [11]:
#inst_dict = dict(zip(unique_insts, all_inst_latlons))
inst_dict = pickle.load(open("institute_latlon.pkl", "rb"))

In [12]:
all_attendee_latlons = np.zeros((len(inst), 2))
flag_nan = np.zeros((len(inst)))
for i in tqdm(range(len(inst))):
    all_attendee_latlons[i] = inst_dict[inst[i]]
    if(np.isnan(inst_dict[inst[i]][0])):
        flag_nan[i] = 1

In [13]:
nan_flag = flag_nan == 0
final_insts = inst[nan_flag]
final_countries = country[nan_flag]
final_latlons = all_attendee_latlons[nan_flag]

In [14]:
def choose_attendees(number_attendees, proportion_north_american=0.9):
    p = np.zeros(len(final_countries))
    number_north_america = len(final_countries[(final_countries == "United States") | (final_countries == "Canada") | (final_countries == "Mexico")])
    number_world = len(final_countries) - number_north_america
    for i in range(len(final_countries)):
        if(final_countries[i] == "United States" or final_countries[i] == "Canada" or final_countries[i] == "Mexico"):
            p[i] = proportion_north_american/number_north_america
        else:
            p[i] = (1.0 - proportion_north_american)/number_world

    choice_attendees = np.random.choice(np.arange(len(final_countries)), number_attendees, p=p)
    return choice_attendees

In [30]:
honolulu_lat = 21.3069
honolulu_lon = -157.8583
ny_lat = 40.7128
ny_lon = -74.0060

In [31]:
def calculate_carbon_footprint_dumb(number_of_people, conference_location, proportion_north_american=0.9):
    choose_attendee_locs = final_latlons[choose_attendees(number_of_people, proportion_north_american=proportion_north_american)]
    all_distances = vc_great_circle(choose_attendee_locs[:,1], choose_attendee_locs[:,0], conference_location[0], conference_location[1])*2 
    
    
    s = 0
    
    print(all_distances)
    for i in tqdm(range(len(all_distances))):
        s += carbon_simple_air_travel(all_distances[i])
    
    return s

In [40]:
a = calculate_carbon_footprint_dumb(2500, [honolulu_lon, honolulu_lat], proportion_north_american=0.9)

[12781.64234128 13665.94641463 15546.80159204 ... 16110.4890962
 15593.12611136 15801.7726237 ]


In [41]:
a/1000

4977.503522554537

In [ ]:
al